# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv

# Import API key
from api_keys import geoapify_key

C:\Users\jhaug\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margate,51.3813,1.3862,14.98,60,57,5.36,GB,1712061578
1,1,ilulissat,69.2167,-51.1000,-2.84,68,40,0.00,GL,1712061484
2,2,waitangi,-43.9535,-176.5597,11.72,93,100,2.05,NZ,1712061471
3,3,meggen,47.0469,8.3747,14.95,53,40,5.14,CH,1712061578
4,4,albany,42.6001,-73.9662,4.49,80,100,1.34,US,1712061258


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
     geo = True,
     tiles = "OSM",
     width = 1000,
     height = 500,
     color = "City",  # generates the side legend
     size = "Humidity"
)

# Display the map plot
# Note to grader: May need to restart kernel to view more than one time; map is only good for one iteration....
map_plot_1

# Note: Leaving x- and y-axes as Longitude and Latitude, respectively, for presentation; not using x and y

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Looking for a place that is between 32 and 77 degrees Farenheit, humidity less than 50% 

narrow_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] > 0) & (city_data_df["Max Temp"] < 25) & (city_data_df["Humidity"] < 50)]
narrow_city_data_df = narrow_city_data_df.dropna(axis = 1)
narrow_city_data_df.reset_index(inplace=True)
#del narrow_city_data_df['index']
narrow_city_data_df.head()

# Drop any rows with null values
narrow_city_data_df.dropna(axis = 0) 

# Display sample data
narrow_city_data_df.head()

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,16,16,constantia,44.1833,28.6500,23.53,39,80,9.83,RO,1712061581
1,35,35,murun-kuren,49.6342,100.1625,3.02,46,10,2.43,MN,1712061585
2,54,54,nurota,40.5614,65.6886,14.60,35,100,3.07,UZ,1712061589
3,58,58,ahar,38.4774,47.0699,16.20,33,100,2.51,IR,1712061590
4,61,61,san patricio,28.0170,-97.5169,23.95,48,60,5.34,US,1712061591


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\jhaug\AppData\Local\Temp\ipykernel_38528\1708204268.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
0,margate,GB,51.3813,1.3862,60,
1,ilulissat,GL,69.2167,-51.1000,68,
2,waitangi,NZ,-43.9535,-176.5597,93,
3,meggen,CH,47.0469,8.3747,53,
4,albany,US,42.6001,-73.9662,80,
...,...,...,...,...,...,...
580,sedalia,US,38.7045,-93.2283,100,
581,alto piquiri,BR,-24.0281,-53.4406,70,
582,juan diaz covarrubias,MX,18.1628,-95.2069,72,
583,yoichi,JP,43.2039,140.7703,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 1  # finding the first hotel 
params = {
    "limit":limit,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    params["categories"] = f'accommodation.hotel'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']}: nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df.head() 


Starting hotel search
margate: nearest hotel: The Reading Rooms
ilulissat: nearest hotel: Best Western Plus Hotel Ilulissat
waitangi: nearest hotel: Hotel Chathams
meggen: nearest hotel: Balm
albany: nearest hotel: No hotel found
stanley: nearest hotel: Hotel 52
port-aux-francais: nearest hotel: Keravel
qaqortoq: nearest hotel: Hotel Qaqortoq
fort bragg: nearest hotel: Airborne Inn Lodging
adamstown: nearest hotel: No hotel found
sao jose da coroa grande: nearest hotel: Hotel do Sol
mochudi: nearest hotel: Rasesa Lodge
college: nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
nyala: nearest hotel: سوني سفاري
lompoc: nearest hotel: Red Roof Inn Lompoc
hithadhoo: nearest hotel: Pebbles Inn
constantia: nearest hotel: Hotel Chérica
new norfolk: nearest hotel: Woodbridge on the Derwent
khandud: nearest hotel: Yamg Homestay
luwingu: nearest hotel: No hotel found
boa vista: nearest hotel: Aipana Plaza Hotel
punta arenas: nearest hotel: Hotel Lacolet
tateyama: nearest hotel: No hotel found
cr

,City,Country,Lat,Lng,Humidity,Hotel Name
0,margate,GB,51.3813,1.3862,60,The Reading Rooms
1,ilulissat,GL,69.2167,-51.1000,68,Best Western Plus Hotel Ilulissat
2,waitangi,NZ,-43.9535,-176.5597,93,Hotel Chathams
3,meggen,CH,47.0469,8.3747,53,Balm
4,albany,US,42.6001,-73.9662,80,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
     geo = True,
     tiles = "OSM",
     width = 1000,
     height = 500,
     color= "City",  # generates the side legend
     size = "Humidity",   # maintaining size of points/markers
     hover_cols = ["Hotel Name","Country"]
)

# Display the map
map_plot_2

# Note: Leaving x- and y-axes as Longitude and Latitude, respectively, for presentation; not using x and y

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)